In [1]:
pip install annoy

     |████████████████████████████████| 646 kB 4.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391638 sha256=5f4c6c4e35931017b6c5ecd70f138d3032bb0c195a9e596223f5c4292be9b6c0
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:
%%capture
# !unzip /content/drive/MyDrive/Datasets/Gallery.zip

!unzip /content/drive/MyDrive/Datasets/Animal_classes_93.zip

In [17]:
# For commands
import os
os.chdir('/content/')
import time
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
# For array manipulation
import numpy as np
import pandas as pd
import pandas.util.testing as tm
# For visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import cv2
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio as io
from pylab import *
from sklearn.manifold import TSNE
#For model performance
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
import joblib
#For model training
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from PIL import Image
from annoy import AnnoyIndex
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
np.random.seed(1337) # for reproducibility
from keras.models import Sequential
from PIL import Image 
import os 
from IPython.display import display
from IPython.display import Image as _Imgd
from glob import glob
import matplotlib.pyplot as plt
from datetime import datetime
from keras.models import load_model

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Populating the interactive namespace from numpy and matplotlib


In [4]:
IMAGE_SIZE = [224, 224]
train_path = '/content/Animal_classes_93/animals/animals'

# VGG16 model imported
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


In [5]:
folders = glob('/content/Animal_classes_93/animals/animals/*')
print("Our training set has "+str(len(folders)) + " classes")

Our training set has 93 classes


In [22]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [23]:
from tensorflow import keras
import tensorflow as tf 
from keras import optimizers


adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [24]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
  monitor='val_loss', min_delta=0.0001, patience=3)

start = datetime.now()
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_path, # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data

model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 32,
    epochs = 20,
    callbacks=[earlystop_callback])

model.save('/content/drive/MyDrive/Datasets/saved_transfer_model.h5')


duration = datetime.now() - start
print("Training completed in time: ", duration)

Found 37544 images belonging to 93 classes.
Found 9378 images belonging to 93 classes.
Epoch 1/20
1173/1173 [==============================] - 670s 570ms/step - loss: 1.4666 - accuracy: 0.7499 - val_loss: 2.5676 - val_accuracy: 0.6510
Epoch 2/20
1173/1173 [==============================] - 666s 568ms/step - loss: 0.9250 - accuracy: 0.8322 - val_loss: 2.4004 - val_accuracy: 0.6945
Epoch 3/20
1173/1173 [==============================] - 666s 568ms/step - loss: 0.7413 - accuracy: 0.8617 - val_loss: 2.8736 - val_accuracy: 0.6839
Epoch 4/20
1173/1173 [==============================] - 664s 566ms/step - loss: 0.6478 - accuracy: 0.8774 - val_loss: 2.6596 - val_accuracy: 0.6870
Epoch 5/20
1173/1173 [==============================] - 665s 567ms/step - loss: 0.5619 - accuracy: 0.8946 - val_loss: 2.8198 - val_accuracy: 0.6984
Training completed in time:  0:55:34.714339


In [25]:
# # MobileNet

# # Mobile model imported
# MobileNetModel = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



# MobileNetModel.save('/content/drive/MyDrive/Datasets/TrialMobileNet_modelV2.h5')

In [26]:
# # Ensemble two models

# keras_model = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/saved_transfer_model.h5', compile=False)
# keras_model._name = 'model1'
# #keras_model2 = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/saved_transfer_modelV2.h5', compile=False)
# keras_model2 = tf.keras.models.load_model('/content/drive/MyDrive/Datasets/TrialMobileNet_modelV2.h5', compile=False)
# keras_model2._name = 'model2'
# models = [keras_model, keras_model2]

# model_input = tf.keras.Input(shape=(224, 224, 3))
# model_outputs = [model(model_input) for model in models]
# ensemble_output = tf.keras.layers.Average()(model_outputs)
# ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)

# ensemble_model.save('/content/drive/MyDrive/Datasets/Ensembled_model.h5')



---



---

#### Here we import the pre-trained model we created and use. 

In [27]:
image_data_with_features_pkl = os.path.join('meta-data-files/','image_data_features.pkl')
image_features_vectors_ann = os.path.join('meta-data-files/','image_features_vectors.ann')

class LoadData:
    """Loading the data from Single/Multiple Folders or form CSV file"""
    def __init__(self):
        pass
    def from_folder(self,folder_list:list): # Enter the Single Folder Path/List of the Folders
        self.folder_list = folder_list
        image_path = []
        for folder in self.folder_list:
            for path in os.listdir(folder):
                image_path.append(os.path.join(folder,path))
        return image_path # Returning list of images
    def from_csv(self,csv_file_path:str,images_column_name:str): # CSV File path with Images path Columns Name
        self.csv_file_path = csv_file_path
        self.images_column_name = images_column_name
        return pd.read_csv(self.csv_file_path)[self.images_column_name].to_list() # Returning list of images

class FeatureExtractor:
    def __init__(self):
        # Use VGG-16 as the architecture and ImageNet for the weight
        #base_model = VGG16(weights='imagenet')
        #base_model = keras.models.load_model('/content/saved_transfer_modelV1.h5')
        base_model = keras.models.load_model('/content/drive/MyDrive/Datasets/Ensembled_model.h5')
        # Customize the model to return features from fully-connected layer
        self.model = Model(inputs=base_model.input, outputs=base_model.get_layer('dense').output)
    def extract(self, img):
        # Resize the image
        img = img.resize((224, 224))
        # Convert the image color space
        img = img.convert('RGB')
        # Reformat the image
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # Extract Features
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)
    def get_feature(self,image_data:list):
        self.image_data = image_data 
        fe = FeatureExtractor()
        features = []
        for img_path in tqdm(self.image_data): # Iterate through images 
            # Extract Features
            try:
                feature = self.extract(img=Image.open(img_path))
                features.append(feature)
            except:
                features.append(None)
                continue
        return features

class Index:
    def __init__(self,image_list:list):
        self.image_list = image_list
        if 'meta-data-files' not in os.listdir():
            os.makedirs("meta-data-files")
        self.FE = FeatureExtractor()
    def start_feature_extraction(self):
        image_data = pd.DataFrame()
        image_data['images_paths'] = self.image_list
        f_data = self.FE.get_feature(self.image_list)
        image_data['features']  = f_data
        image_data = image_data.dropna().reset_index(drop=True)
        
        image_data.to_pickle(image_data_with_features_pkl)
        print("Image Meta Information Saved: [meta-data-files/image_data_features.pkl]")
        return image_data
    def start_indexing(self,image_data):
        self.image_data = image_data
        f = len(image_data['features'][0]) # Length of item vector that will be indexed
        t = AnnoyIndex(f, 'euclidean')
        for i,v in tqdm(zip(self.image_data.index,image_data['features'])):
            t.add_item(i, v)
        t.build(100) # 100 trees
        print("Saved the Indexed File:"+"[meta-data-files/image_features_vectors.ann]")
        
        t.save(image_features_vectors_ann)
    def Start(self):
        if len(os.listdir("meta-data-files/"))==0:
            data = self.start_feature_extraction()
            self.start_indexing(data)
        else:
            print("Metadata and Features are already present, Do you want Extract Again? Enter yes or no")
            flag  = str(input())
            if flag.lower() == 'yes':
                data = self.start_feature_extraction()
                self.start_indexing(data)
            else:
                print("Meta data already Present, Please Apply Search!")
                print(os.listdir("meta-data-files/"))

class SearchImage:
    def __init__(self):
        self.image_data = pd.read_pickle(image_data_with_features_pkl)
        self.f = len(self.image_data['features'][0])
    def search_by_vector(self,v,n:int):
        self.v = v # Feature Vector
        self.n = n # number of output 
        u = AnnoyIndex(self.f, 'euclidean')
        u.load(image_features_vectors_ann) # super fast, will just mmap the file
        index_list = u.get_nns_by_vector(self.v, self.n) # will find the 10 nearest neighbors
        return dict(zip(index_list,self.image_data.iloc[index_list]['images_paths'].to_list()))
    def get_query_vector(self,image_path:str):
        self.image_path = image_path
        img = Image.open(self.image_path)
        fe = FeatureExtractor()
        query_vector = fe.extract(img)
        return query_vector
    def plot_similar_images(self,image_path:str):
        self.image_path = image_path
        query_vector = self.get_query_vector(self.image_path)
        img_list = list(self.search_by_vector(query_vector,16).values())
        # Visualize the result
        axes=[]
        fig=plt.figure(figsize=(20,15))
        for a in range(4*4):
            axes.append(fig.add_subplot(4, 4, a+1))  
            plt.axis('off')
            plt.imshow(Image.open(img_list[a]))
        fig.tight_layout()
        fig.suptitle('Similar Result Found', fontsize=22)
        plt.show(fig)
    def get_similar_images(self,image_path:str,number_of_images:int):
        self.image_path = image_path
        self.number_of_images = number_of_images
        query_vector = self.get_query_vector(self.image_path)
        img_dict = self.search_by_vector(query_vector,self.number_of_images)
        return img_dict

In [ ]:
# Unzip the gallery here:
%%capture
!unzip /content/drive/MyDrive/Datasets/galler.zip

In [ ]:
# load the Images from the Folder (You can also import data from multiple folders in python list type)
image_list = LoadData().from_folder(folder_list = ['galler'])


In [ ]:
# For Faster Serching we need to index Data first, After Indexing all the meta data stored on the local path
Index(image_list).Start()


In [ ]:
# for searching, you need to give the image path and the number of the similar image you want
top10 = SearchImage().get_similar_images(image_path='/content/A1 (7).jpg',number_of_images=10)
top10

In [ ]:
i =1
for x in top10:
  
  print("Top "+str(i)+ " image")
  i=i+1
  img = mpimg.imread(top10[x])
  imgplot = plt.imshow(img)
  plt.show()



---



---

Ignore everything below this: